In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications import InceptionResNetV2
from six.moves import cPickle
# from inception_resnet_v2 import InceptionResNetV2
from keras.layers import Flatten, Dense, AveragePooling2D,Dropout, GlobalAveragePooling2D
from keras.models import Sequential
from keras.datasets import cifar10
from keras.models import Model, load_model
from keras.optimizers import rmsprop, SGD
import cv2
import numpy as np
import os
from keras.utils import to_categorical 
import matplotlib.pyplot as plt

In [2]:
def ResNet():
    base_model = ResNet50(include_top=False, weights='imagenet')

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(512, activation='relu')(x)
    # and a logistic layer -- 10 classes for Tire Defects
    predictions = Dense(y_train_categorical.shape[1], activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy')
    model.fit(x_train, y_train_categorical)
    
    layer_num = len(model.layers)
    for layer in base_model.layers[:175]:
        layer.trainable = False
    for layer in base_model.layers[175:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train_categorical, epochs=5, shuffle=True)
    
    return model, history

In [4]:
def Reshape(data):
    large_img = cv2.resize(img, dsize=(500, 700), interpolation=cv2.INTER_CUBIC)
    return large_img

In [97]:
PIXEL = 300
def Datasetprep():
    # Data preparation
    path = './'
    x_train = []
    y_train = []
    for folder in os.listdir(path):
        folderPath = path + folder
        if folderPath[-3:] == '.py' or folderPath[-5:] == '.json':
            continue
        for image_name in os.listdir(folderPath):
            img = cv2.imread(os.path.join(folderPath,image_name))
            resizedImg = cv2.resize(img,(PIXEL,PIXEL))
            x_train.append(np.array(resizedImg))
            y_train.append(int(folder))
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    y_train_categorical = to_categorical(y_train)
    return x_train, y_train_categorical

In [98]:
x_train, y_train_categorical = Datasetprep()
print(x_train.shape, y_train_categorical.shape)

(186, 300, 300, 3) (186, 14)


In [5]:
# For Training the Model
# WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1349: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dimmodel, history = ResNet()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/1
182/182 [==============================] - 74s 409ms/step - loss: 5.5483
Train on 127 samples, validate on 55 samples
Epoch 1/5
127/127 [==============================] - 77s 610ms/step - loss: 1.7743 - acc: 0.5984 - val_loss: 1.2139 - val_acc: 0.7091
Epoch 2/5
127/127 [==============================] - 76s 595ms/step - loss: 0.9820 - acc: 0.7087 - val_loss: 1.6695 - val_acc: 0.7091
Epoch 3/5
127/127 [==============================] - 75s 593ms/step - loss: 1.3164 - acc: 0.6063 - val_loss: 1.2378 - val_acc: 0.6909
Epoch 4/5
127/127 [==============================] - 75s 592ms/step - loss: 0.8113 - acc: 0.7165 - val_loss: 1.5140 - val_acc: 0.7091
Epoch 5/5
127/127 [==============================] - 76s 598ms/step - loss: 0.9307 - acc: 0.7244 - val_loss: 1.6384 - val_acc: 0.6727


In [6]:
# Saving the Weights of the Model
# model.save_weights('Resnt_weights.h5')

In [6]:
def show_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'test_accuracy'], loc='best')
    plt.show()

In [7]:
def Prediction(x_train, y_train):
    predictions = model.predict(x_train)
    print(show_history(history))
    print(predictions)
    print(x_train.shape,y_train_categorical.shape)
    
    return predictions

In [13]:
def ProcessingImage(image):
    
    test = []
    resizedImage = cv2.resize(image, dsize=(PIXEL, PIXEL))
    test.append(resizedImage)
    test = np.array(test)
    
    return test

In [92]:
import json
import os

def json_write(idx,url1,url2,url3):
    arr = [idx]
    arrn = []
    for i in range(0,13):
        x = arr.__contains__(i)
        if x==False:
            arrn.append(i)

    data = {}  
    data['tyres'] = []  
    data['num'] = []
    data['url'] = []
    data['url'].append({
        'url1':url1,
        'url2':url2,
        'url3':url3
    })
    if(arr.__contains__(0)):
            data['num'].append({  
                'title': '0',
                    })
    for i in arr:
        
        '''if i == 0:
                                        data['tyres'].append({  
                                        'title': 'good',
                                            })'''
        if i == 1:
            data['tyres'].append({  
                'title': 'Tread wear indicator',
                'description': 'present'
            })
        if i == 2:
            data['tyres'].append({  
                'title': 'Off center snacking',
                'description': 'present'
            })
        if i == 3:
            data['tyres'].append({  
                'title': 'Bare/Lightness',
                'description': 'present'
            })
        if i == 4:
            data['tyres'].append({  
                'title': 'Blister',
                'description': 'present'
            })
        if i == 5:
            data['tyres'].append({  
                'title': 'Foreign Material',
                'description': 'present'
            })
        if i == 6:
            data['tyres'].append({  
                'title': 'Open cord',
                'description': 'present'
            })
        if i == 7:
            data['tyres'].append({  
                'title': 'Exposed Cord',
                'description': 'present'
            })
        if i == 8:
            data['tyres'].append({  
                'title': 'Crown deformation',
                'description': 'present'
            })
        if i == 9:
            data['tyres'].append({  
                'title': 'Bladder crease',
                'description': 'present'
            })
        if i == 10:
            data['tyres'].append({  
                'title': 'Separation',
                'description': 'present'
            })
        if i == 11:
            data['tyres'].append({  
                'title': 'Line Air',
                'description': 'present'
            })
        if i == 12:
            data['tyres'].append({  
                'title': 'Over Shaping',
                'description': 'present'
            })
        if i == 13:
            data['tyres'].append({  
                'title': 'Scorch Rubber',
                'description': 'present'
            })
    if(arrn.__contains__(0)):
            data['num'].append({  
                'title': '1',
                    })
    for i in arrn:
        
        if i == 1:
            data['tyres'].append({  
                'title': 'Tread wear indicator',
                'description': 'null'
            })
        if i == 2:
            data['tyres'].append({  
                'title': 'Off center snacking',
                'description': 'null'
            })
        if i == 3:
            data['tyres'].append({  
                'title': 'Bare/Lightness',
                'description': 'null'
            })
        if i == 4:
            data['tyres'].append({  
                'title': 'Blister',
                'description': 'null'
            })
        if i == 5:
            data['tyres'].append({  
                'title': 'Foreign Material',
                'description': 'null'
            })
        if i == 6:
            data['tyres'].append({  
                'title': 'Open cord',
                'description': 'null'
            })
        if i == 7:
            data['tyres'].append({  
                'title': 'Exposed Cord',
                'description': 'null'
            })
        if i == 8:
            data['tyres'].append({  
                'title': 'Crown deformation',
                'description': 'null'
            })
        if i == 9:
            data['tyres'].append({  
                'title': 'Bladder crease',
                'description': 'null'
            })
        if i == 10:
            data['tyres'].append({  
                'title': 'Separation',
                'description': 'null'
            })
        if i == 11:
            data['tyres'].append({  
                'title': 'Line Air',
                'description': 'null'
            })
        if i == 12:
            data['tyres'].append({  
                'title': 'Over Shaping',
                'description': 'null'
            })
        if i == 13:
            data['tyres'].append({  
                'title': 'Scorch Rubber',
                'description': 'null'
            })
    

    with open('data.json', 'w') as outfile:  
        json.dump(data, outfile)

# arr = [idx]
# arrn = []
# for i in range(0,13):
#     x = arr.__contains__(i)
#     if x==False:
#         arrn.append(i)
# print(idx)
# print(arrn)

#path = os.path.dirname(__file__)      
#url1 = "http://bitsotg.com:7878" +"/image1.jpg"
#url2 = "http://bitsotg.com:7878" +"/image4.jpg"
#url3 = "http://bitsotg.com:7878" +"/image7.jpg"
# print(url)
#json_write(arr,arrn,url,url,url)



In [99]:
url1 = url2 = url3 = ""
def Predict(IMG_PATH, model):
    test_img = cv2.imread(IMG_PATH)
    
    if test_img is None or test_img.shape[0] == 0:
        return -1
    cv2.imshow('Test Image', test_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    url = url1 = url2 = url3 = "http://192.168.43.196:7879/"+ IMG_PATH[2:]
    print(url1)
    processed_img = ProcessingImage(test_img)
    predictions_resnet = model.predict(processed_img)
    print(predictions_resnet)
    idx = FindingIndex(predictions_resnet)
    json_write(idx,url,url,url)
    return idx

In [100]:
def FindingIndex(predictions_resnet):
    idx = -1
    idx2 = -1
    idx3 = -1
    maxVal = 0
    secondLargest = 0
    ThirdLargest = 0
    k = 0
    for i in range(predictions_resnet.shape[0]):
        for j in range(predictions_resnet.shape[1]):

            if predictions_resnet[i][j] > maxVal:
                idx = k
                ThirdLargest = secondLargest
                secondLargest = maxVal
                maxVal = predictions_resnet[i][j]
            elif predictions_resnet[i][j] > secondLargest:
                ThirdLargest = secondLargest
                secondLargest = predictions_resnet[i][j]
                idx2 = k
            elif predictions_resnet[i][j] > ThirdLargest:
                ThirdLargest = predictions_resnet[i][j]
                idx3 = k
            k += 1
    print("First Largest index Prediction is " + str(idx)+" With Prediction "+ str(maxVal))
    print("Second Closest index Prediction is " + str(idx2)+" With Prediction "+ str(secondLargest))
    print("Third Largest index Prediction is " + str(idx3)+" With Prediction "+ str(ThirdLargest))
    return idx

In [ ]:
''' Some Sample Paths for testing the Prediction of the Model
    IMG_PATH = "./Images/4/bmSwAtC.png"
    IMG_PATH = "./Images/13/JFPpxUg.png"
    IMG_PATH = "./Images/0/03SINCERA-845.png"
    IMG_PATH= "./Images/11/UbQ02HU.png"
    IMG_PATH = "./Images/0/12x.png"
    IMG_PATH = "./Images/3/download (2).jpg"
    IMG_PATH = "./Images/12/WhatsApp Image 2019-03-03 at 2.33.42 AM.jpeg"
    IMG_PATH = "./Images/8/Bulge-on-Tire-Sidewall.png"
    IMG_PATH = "./Images/12/images.jpg" '''
# path = IMG_PATH
# Model Binded Together
x_train, y_train_categorical = Datasetprep()
model, history = ResNet()
# model.save_weights('Resnet_weights.h5')

#Saving the Prev Model
# saveModel(model)
IMG_PATH = "./Images/0/03SINCERA-845.png"
idx = Predict(IMG_PATH,model)
# print(url)
#Reading the Existing Model
# new_model = ReadModel()
# print("New One")
# print(Predict(IMG_PATH, new_model))

Epoch 1/1
186/186 [==============================] - 79s 427ms/step - loss: 4.8657
Train on 130 samples, validate on 56 samples
Epoch 1/5
130/130 [==============================] - 79s 608ms/step - loss: 1.9061 - acc: 0.5769 - val_loss: 0.9396 - val_acc: 0.7143
Epoch 2/5
130/130 [==============================] - 80s 616ms/step - loss: 1.1362 - acc: 0.6000 - val_loss: 1.2730 - val_acc: 0.6964
Epoch 3/5
130/130 [==============================] - 81s 621ms/step - loss: 0.8399 - acc: 0.7692 - val_loss: 1.5124 - val_acc: 0.6607
Epoch 4/5
130/130 [==============================] - 77s 593ms/step - loss: 0.6813 - acc: 0.7769 - val_loss: 1.9118 - val_acc: 0.6964
Epoch 5/5
130/130 [==============================] - 83s 635ms/step - loss: 0.7082 - acc: 0.7692 - val_loss: 1.5235 - val_acc: 0.6964


In [79]:
def saveModel(model):
    model.save('my_model.h5')
    return

def ReadModel():
    
    model = load_model('my_model.h5')
    return model

8
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12]
http://192.168.43.196:7879/Images/8/Bulge-on-Tire-Sidewall.png
